### Import Statements

In [ ]:
import requests
import pandas as pd
import json
import time
import os

### Retrieve API Key

In [ ]:
# to use this notebook, fill in values for your api user and api key
#####User Input#####
api_user = "INSERT_API_USER"
api_pass = "INSERT_API_KEY|1"
#####User Input#####

headers = {"content-type": "application/json"}

### Get API Key Profiles

In [ ]:
intro_url = "https://api.securevan.com/v4/apiKeyProfiles/"
response = (
    requests.get(
    intro_url,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
        ).json()
    )

In [ ]:
response

### Get Export Job Types

In [ ]:
job_types_url = "https://api.securevan.com/v4/exportJobTypes"

job_types = (
    requests.get(job_types_url,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
        ).json()
    )

In [ ]:
job_types

### Get Saved Lists

In [ ]:
saved_lists_url = "https://api.securevan.com/v4/savedLists"

query_string = {"maxDoorCount":"1000","maxPeopleCount":"2000"}

saved_lists = (
    requests.request(
        "GET",
        saved_lists_url,
        headers=headers,
        auth=requests.auth.HTTPBasicAuth(api_user, api_pass),
        params=query_string
            ).json()
    )

In [ ]:
saved_lists

### Check SavedList is in ID

In [ ]:
#####User Input#####
saved_list_id = 'INSERT_LIST_ID_AS_INT'
#####User Input#####

if (saved_list_id in set([item['savedListId'] for item in saved_lists['items']])):
    print('Valid Saved List')
else:
    print('Invalid Saved List')

### Create Export Job

After creating the export job, there are two options on receiving the download url. The first is through repeateduly calling the API, at set intervals, to check if the job is complete. Once it is, the download link will be printed. The second is through using a webhook which will post the job details once it is complete, which our function will then get to retrieve the downlaod link. Only one of the following implementations are needed. 

In [ ]:
#####User Input#####
webhook_url = "INSERT_WEBHOOK_URL"
job_type = 'INSERT_JOB_TYPE_AS_INT'
#####User Input#####

url = "https://api.securevan.com/v4/exportJobs"

payload = {
    "savedListId": saved_list_id,
    "type": job_type,
    "webhookUrl": webhook_url
}

response = (requests.request(
    "POST", 
    url, 
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass),
    json=payload, 
    headers=headers)
           ).json()

In [ ]:
response

### Pulling - Check export status retrieve download URL

In [ ]:
export_job_id = response["exportJobId"]

url = f"https://api.securevan.com/v4/exportJobs/{export_job_id}"

while True:
    response = requests.request(
        "GET", 
        url, 
        headers=headers, 
        auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
            ).json()
    
    if response["status"] != "Completed":
        print(f"Job Status: {response['status']}")
        time.sleep(5)
        
    else:
        print(f"Job Status: {response['status']}")
        print(f"File at {response['downloadUrl']}")
        break

### Webhook - Check export status and retrieve download URL

In [ ]:
headers = {"content-type": "application/json"}
response = requests.get(
    webhook_url,
    headers=headers
        ).json()

print(f"Job Status: {response['status']}")
print(f"File at {response['downloadUrl']}")

### Download CSV and load into panda frame

In [ ]:
#####User Input#####
file_name = "INSERT_FILE_NAME"
#####User Input#####

resp = requests.get(response['downloadUrl'])
with open(file_name, "wb") as f:
    f.write(resp.content)
frame = pd.read_csv(fname)

In [ ]:
frame